In [1]:
## LIBRARY CONFIGURATION
%load_ext autoreload
%autoreload 2
    
from cfg import *
sys.path.append('libraries')

from libraries.utils import *

import libraries.AllenRegistration_beta as AR_Lib

In [2]:
## Function to be called in the parallel-processing loop, that treats each experiment-to-be-registered as a distinct Python dictionary

def fast_section_registration(section_id, AllenData, section_dict, experiment_dir):

    input_dict = {}
    input_dict[section_id] = AR_Lib.ImageRegistration(AllenData, section_dict[section_id], experiment_dir)
    input_dict[section_id].register_section()

    return input_dict

In [3]:
## SETTING THE PATHS TO BE USED BY THE PIPELINE

current_path = './' # to be replaced with os.getcwd()

# Choose directories for pre-process atlas files, flatmap files, input dataset and other code repositories
atlas_dir = 'atlas_files'
sys.path.append(atlas_dir)

flatmapper_dir = 'cortical_coordinates'
sys.path.append(flatmapper_dir)

dataset_dir = os.path.join(current_path, 'Datasets') # To be replaced with 'Datasets'
sys.path.append(dataset_dir)

In [4]:
# Choose a directory to save your output

img_output_dir = os.path.join(current_path,'Saves')
save_figure_dir = os.path.join(current_path,'Figures')

# Configuration parameters
resolution = 10 # Intiializing for 10 um resolution


In [ ]:
#!!!! Parallel Approach !!!!!!!!!!!!!!!!!!!!

AR_dict = OrderedDict()
experiment_list = os.listdir(dataset_dir)


# Initializing the AllenData constructor which acts as a fake-interface to each class generated for each section-to-be-registered: 
# the classes take all allen-data-related information from it

AllenData = AR_Lib.AllenDataStore(flatmapper_dir, atlas_dir, resolution)

for experiment_id in experiment_list:
    
    experiment_dir = os.path.join(dataset_dir, experiment_id)
    section_dict = AR_Lib.get_section_info(experiment_dir)
    if section_dict == -1: continue

    partial_process_item = functools.partial(fast_section_registration, AllenData=AllenData, section_dict=section_dict,
                                             experiment_dir=experiment_dir)

    start_time = time.time()
    AR_dict[experiment_id] = Parallel(n_jobs=2)(delayed(partial_process_item)(section_id) for section_id in section_dict.keys())
    end_time = time.time()
    print(f'{experiment_id} was registered in {np.round((end_time - start_time) / 60).astype(int)} minutes')

    # AR_dict[experiment_id] = []
    # for section_id in section_dict.keys():
    #    AR_dict[experiment_id].append(partial_process_item(section_id))
    #    break
    
    # The command below is required to reformat the list returned by the for-loop as a dictionary containing the section_ids and their registered section images
    AR_dict[experiment_id] = reformat_as_dictionary(AR_dict[experiment_id])
    


### The following cell illustrates how to retrieve the flatmap coordinates and respective annotation for a given set of pixel coordinates belonging to a registered section image

In [12]:
# !! Warning !! The experiment id and section id that I am using here come from our BDA047 registered data. 
#  Obviously those are placeholders and as such you insert your dictionary keys there corresponding to your registered images
# The pixel coordinates (2240, 8) are also arbitrary

# To avoid the long name of the nested dictionary, I create a temporary pointer to use for calling 
# the functions related to the class instance of that registered section
tmp_pointer = AR_dict['BDA047']['BDA047_010x_114_MinProj_cropped_downsampled5_s001']

# Information retrieval for corresponding voxels
corr_vox = tmp_pointer.give_me_corresponding_voxels([2240, 8])
vox_annotation = AllenData.give_me_the_allen_annotation(corr_vox)
print(f'The corresponding voxel coordinates are: {corr_vox},\nwith annotation {vox_annotation}')

# Information retrieval for corresponding flatmap points
corr_flt_pts = tmp_pointer.give_me_flatmap_points([2240, 8])
flt_annotation = AllenData.give_me_the_allen_annotation(corr_flt_pts)
print(f'The corresponding flatmap coordinates are: {corr_flt_pts},\nwith annotation {flt_annotation}')


The corresponding voxel coordinates are: [ 423  119 1008],
with annotation [background]
Error! The coordinates must have 2 or 3 elements corresponding to flatmap or voxel coordinates!
The corresponding flatmap coordinates are: [[-1 -1]],
with annotation -1
